# Problem Statement

Extracting information from legal document

# 1)- Importing key Modules

In [1]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
import warnings
warnings.filterwarnings('ignore')

In [2]:
import string 
import PyPDF2 as p2
import numpy as np 
import pickle
import pandas as pd 

#For Visuals
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import seaborn as sns
sns.set(rc={'figure.figsize':(15,9)})
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# NLTK Stop words
import nltk  # for text manipulation 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [5]:
import re    # for regular expressions 
import spacy # another text processing library like nltk
import en_core_web_md
nlp = en_core_web_md.load()

In [6]:
%reload_ext version_information
%version_information pandas,numpy, nltk, seaborn, matplotlib, re, PyPDF2,gensim, spacy

Software,Version
Python,3.7.4 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
IPython,7.8.0
OS,Darwin 19.0.0 x86_64 i386 64bit
pandas,0.25.1
numpy,1.17.2
nltk,3.4.5
seaborn,0.9.0
matplotlib,3.1.1
re,2.2.1
PyPDF2,1.26.0


I personally like Spacy more than nltk. But, I never take sides so, I ll use both where they come handy

# 2)- Loading data

- 1st from pdf to text
- 2ndly text to proper data

### Code 1: for converting generic text from pdf to text

In [7]:
# extract content of given pdf file

def getpdf(pdffile):
    myfile=p2.PdfFileReader(pdffile)
    
    # create text file that will contain given pdf 's data
    with open('document.txt', 'w') as pdf_output:
        for page in range(myfile.getNumPages()):
            data= myfile.getPage(page).extractText()
            pdf_output.write(data)
            
            
            # read data and cleaning as we have written pdf data into text file
            
            with open('document.txt' , 'r') as mypdfcontent:
                return mypdfcontent.read() # this will return text file and replace all blank spaces

In [8]:
content=getpdf('contract.pdf')

### Code 2: for converting generic text from pdf to text

In [9]:
import PyPDF2
pdffileobj=open('contract.pdf','rb')
pdfreader=PyPDF2.PdfFileReader(pdffileobj)
x=pdfreader.numPages
pageobj=pdfreader.getPage(x-1)
text=pageobj.extractText()

file1=open(r"contract.txt","a")
file1.writelines(text)
file1.close()

### Code 3:for converting generic text from pdf to text

In [10]:
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

In [11]:
"""

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 
   
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        if fileExtension == "pdf":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + pdf + ".txt"
            textFile = open(textFilename, "w") #make text file
            textFile.write(text) #write text to text file
			#textFile.close

pdfDir = "/Users/hassansherwani/Documents/paper"
txtDir = "/Users/hassansherwani/Documents/paper"
convertMultiple(pdfDir, txtDir)

"""

'\n\n#converts pdf, returns its text content as a string\ndef convert(fname, pages=None):\n    if not pages:\n        pagenums = set()\n    else:\n        pagenums = set(pages)\n\n    output = StringIO()\n    manager = PDFResourceManager()\n    converter = TextConverter(manager, output, laparams=LAParams())\n    interpreter = PDFPageInterpreter(manager, converter)\n\n    infile = file(fname, \'rb\')\n    for page in PDFPage.get_pages(infile, pagenums):\n        interpreter.process_page(page)\n    infile.close()\n    converter.close()\n    text = output.getvalue()\n    output.close\n    return text \n   \ndef convertMultiple(pdfDir, txtDir):\n    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in \n    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory\n        fileExtension = pdf.split(".")[-1]\n        if fileExtension == "pdf":\n            pdfFilename = pdfDir + pdf \n            text = convert(pdfFilename) #get string of text content of pdf\n   

### using Pandas for data loading vs open()

In [12]:
df=pd.read_table('xxx.txt',names=['text'])

In [13]:
df.head()

,text
0,TRADEMARK AND DOMAIN NAME AGREEMENT
1,This agreement (the “Agreement”) is by and bet...
2,"(“Party”) and Eclipse Foundation, Inc. (“Eclip..."
3,201[__] (the “Effective Date”).
4,"WHEREAS, Party is the owner of certain tradema..."


In [14]:
df.shape

(176, 1)

In [15]:
df.text[:5]

0                 TRADEMARK AND DOMAIN NAME AGREEMENT 
1    This agreement (the “Agreement”) is by and bet...
2    (“Party”) and Eclipse Foundation, Inc. (“Eclip...
3                     201[__] (the “Effective Date”). 
4    WHEREAS, Party is the owner of certain tradema...
Name: text, dtype: object

In [16]:
df.text[170]

'Page 5 of 5 '

pandas is more useful if we had more features to work with and it could form a table. Pandas is more for dataframes and here, we have only generic text data with unstructured patterns and there is no point in using pandas.

In [17]:
with open('xxx.txt') as f:
    docu = f.read().splitlines()

In [18]:
# first 10 lines
docu[:10]

['TRADEMARK AND DOMAIN NAME AGREEMENT ',
 '',
 'This agreement (the “Agreement”) is by and between ____________________________ ',
 '',
 ' ',
 ' ',
 '(“Party”) and Eclipse Foundation, Inc. (“Eclipse”) and is effective as of [______] [_____],  ',
 '201[__] (the “Effective Date”). ',
 ' ',
 '']

# 3)-Data preprocessing

In [19]:
clean_stage1=[i.replace('\xe2\x80\x9c','') for i in docu]

In [20]:
print(clean_stage1[:10])

['TRADEMARK AND DOMAIN NAME AGREEMENT ', '', 'This agreement (the “Agreement”) is by and between ____________________________ ', '', ' ', ' ', '(“Party”) and Eclipse Foundation, Inc. (“Eclipse”) and is effective as of [______] [_____],  ', '201[__] (the “Effective Date”). ', ' ', '']


In [21]:
clean2=[i.replace('\xe2\x80\x9d','') for i in clean_stage1 ]
clean3=[i.replace('\xe2\x80\x99s','') for i in clean2 ]

In [22]:
clean4 = [x for x in clean3 if x != ' ']
clean5 = [x for x in clean4 if x != '']

In [23]:
final_clean=[re.sub("[^a-zA-Z]+", " ", s) for s in clean5]

In [24]:
final_clean[:15]

['TRADEMARK AND DOMAIN NAME AGREEMENT ',
 'This agreement the Agreement is by and between ',
 ' Party and Eclipse Foundation Inc Eclipse and is effective as of ',
 ' the Effective Date ',
 'WHEREAS Party is the owner of certain trademarks identified in Exhibit A the ',
 ' Trademarks and of certain domain names identified in Exhibit A the Domain Names ',
 'WHEREAS Party is desirous of Eclipse to initiate a project or working group with a ',
 'name based on the Trademark the Project Initiation ',
 'WHEREAS to accommodate the Project Initiation Party desires to transfer all of Party s ',
 'rights title and interest in and to the Trademarks to Eclipse and to thereafter have certain ',
 'continuing usage rights of the Trademarks ',
 'WHEREAS to accommodate the Project Initiation the Parties may mutually agree to ',
 'have any related Domain Names initially redirect to a URL designated by Eclipse and that the ',
 'Domain Names are subsequently transferred to Eclipse ',
 'In consideration of 

These lines look more aligned in comparision to original document

In [25]:
docu[:15]

['TRADEMARK AND DOMAIN NAME AGREEMENT ',
 '',
 'This agreement (the “Agreement”) is by and between ____________________________ ',
 '',
 ' ',
 ' ',
 '(“Party”) and Eclipse Foundation, Inc. (“Eclipse”) and is effective as of [______] [_____],  ',
 '201[__] (the “Effective Date”). ',
 ' ',
 '',
 'WHEREAS, Party is the owner of certain trademarks identified in Exhibit A (the ',
 '',
 '“Trademarks”) and of certain domain names identified in Exhibit A (the “Domain Names”); ',
 '',
 ' ']

#### Cleaning the data

using tokenization

In [26]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [27]:
chars=list(set(docu))

data_size, vocab_size = len(docu), len(chars)
print ("Contract data has chars", data_size)
print ("Contract data has unique chars", vocab_size)

Contract data has chars 412
Contract data has unique chars 173


In [28]:
type(docu)

list

#### Tokenize Sentences 

In [29]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove extra characters
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)

In [30]:
# Convert to list
#data = docu.tolist()
data_words = list(sent_to_words(docu))
print(data_words[:1])

[['trademark', 'and', 'domain', 'name', 'agreement']]


#### Build the Bigram, Trigram Models,Postag and Lemmatize

In [31]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [32]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = en_core_web_md.load()
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [33]:
df_data_ready = pd.DataFrame(data_ready)

In [34]:
df_data_ready.head()

,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,trademark,domain,name,agreement,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,agreement,agreement,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [35]:
df_data_ready.shape

(412, 190)

In [36]:
import pickle
df_data_ready.to_pickle('data_ready.pkl')

# 4)- Model Buidling

Topic model for keyword extraction

### 4.1)- Create Dictionary

In [37]:
#bag of words
id2word = corpora.Dictionary(data_ready)

In [38]:
print(len(id2word))

283


In [39]:

count = 0
for k, v in id2word.iteritems():
    print(k, v, id2word.dfs[k])
    count += 1
    if count > 10:
        break

0 agreement 24
1 domain 15
2 name 21
3 trademark 30
4 effective 9
5 date 11
6 certain 4
7 exhibit 5
8 identify 3
9 owner 3
10 party 32


we could use a filter but data is very small

### 4.2)-Create Corpus: Term Document Frequency

In [40]:
docu[10]

'WHEREAS, Party is the owner of certain trademarks identified in Exhibit A (the '

In [41]:
data_ready[10]

['party', 'owner', 'certain', 'trademark', 'identify', 'exhibit']

In [42]:
corpus = [id2word.doc2bow(text) for text in data_ready]
corpus[10] # 10 as an example

[(3, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]

In [43]:
print(len(corpus))

412


In [44]:
bow_doc_10 = corpus[10]

for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], 
                                                     id2word[bow_doc_10[i][0]], 
                                                     bow_doc_10[i][1]))

Word 3 ("trademark") appears 1 time.
Word 6 ("certain") appears 1 time.
Word 7 ("exhibit") appears 1 time.
Word 8 ("identify") appears 1 time.
Word 9 ("owner") appears 1 time.
Word 10 ("party") appears 1 time.


### 4.3)- LDA

**Parameters:**

- corpus ({iterable of list of (int, float), scipy.sparse.csc}, optional) – Stream of document vectors or sparse matrix of shape (num_terms, num_documents). If not given, the model is left untrained (presumably because you want to call update() manually).
- num_topics (int, optional) – The number of requested latent topics to be extracted from the training corpus.
- id2word ({dict of (int, str), gensim.corpora.dictionary.Dictionary}) – Mapping from word IDs to words. It is used to determine the vocabulary size, as well as for debugging and topic printing.
- distributed (bool, optional) – Whether distributed computing should be used to accelerate training.
- chunksize (int, optional) – Number of documents to be used in each training chunk.
- passes (int, optional) – Number of passes through the corpus during training.
- update_every (int, optional) – Number of documents to be iterated through for each update. Set to 0 for batch learning, > 1 for online iterative learning.
- alpha ({numpy.ndarray, str}, optional) – Can be set to an 1D array of length equal to the number of expected topics that expresses our a-priori belief for the each topics’ probability. Alternatively default prior selecting strategies can be employed by supplying a string:

’asymmetric’: Uses a fixed normalized asymmetric prior of 1.0 / topicno. ’auto’: Learns an asymmetric prior from the corpus (not available if distributed==True).

eta ({float, np.array, str}, optional) –

A-priori belief on word probability, this can be:

scalar for a symmetric prior over topic/word probability, vector of length num_words to denote an asymmetric user defined probability for each word, matrix of shape (num_topics, num_words) to assign a probability for each word-topic combination, the string ‘auto’ to learn the asymmetric prior from the data.

In [45]:
lda_model = gensim.models.LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, passes=2, workers=2)

In [46]:
lda_model.save('topic_model')

In [47]:
from gensim.models import LdaModel
lda_model = LdaModel.load('topic_model')

In [48]:
from pprint import pprint

#The trained topics (keywords and weights)
pprint(lda_model.print_topics())

[(0,
  '0.071*"trademark" + 0.062*"party" + 0.027*"eclipse" + 0.027*"title" + '
  '0.021*"assignment" + 0.019*"name" + 0.019*"transfer" + 0.019*"date" + '
  '0.018*"domain" + 0.018*"interest"'),
 (1,
  '0.078*"agreement" + 0.041*"party" + 0.027*"page" + 0.024*"write" + '
  '0.020*"eclipse" + 0.017*"waiver" + 0.014*"name" + 0.012*"independent" + '
  '0.012*"contractor" + 0.011*"term"'),
 (2,
  '0.034*"trademark" + 0.028*"provision" + 0.023*"delivery" + 0.022*"material" '
  '+ 0.022*"document" + 0.021*"notice" + 0.021*"execution" + 0.013*"guideline" '
  '+ 0.012*"arise" + 0.012*"demand"'),
 (3,
  '0.055*"agreement" + 0.034*"party" + 0.032*"term" + 0.031*"condition" + '
  '0.029*"waiver" + 0.016*"interest" + 0.016*"require" + 0.015*"write" + '
  '0.014*"title" + 0.014*"law"'),
 (4,
  '0.069*"name" + 0.057*"domain" + 0.037*"party" + 0.031*"trademark" + '
  '0.026*"remedy" + 0.018*"include" + 0.017*"date" + 0.017*"transfer" + '
  '0.017*"effective" + 0.014*"agreement"')]


# 5)-Evaluating model

### 5.1)- Perplexity

Lower is better

In [49]:
# Compute Perplexity:a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -5.805305456585416


### 5.2)-Coherence Score

Higher is better

In [50]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5263038663720127


### 5.3- Visualization

In [51]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [52]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)

In [53]:
pyLDAvis.save_html(vis, 'topics_LDApyvis.html')